<a href="https://colab.research.google.com/github/spatank/CIS-545/blob/master/project_SPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive

# mount google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
os.chdir('gdrive/MyDrive/CIS545_2020/Project')
!ls

adj.mat  AED_matrices.npy  AED_matrices.npz  project_SPP.ipynb


In [47]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import savemat
import networkx as nx

In [4]:
vars = np.load('AED_matrices.npz', allow_pickle = True)
for item in vars.files:
  print(item)

arr_0
arr_1
arr_2


In [5]:
total_adj = vars['arr_0']
time_var_adj = vars['arr_1']
med_names = vars['arr_2']

In [38]:
def transpose(matrix):
    num_lists_in_transpose = len(matrix[0])
    transpose_matrix = [] # initialize empty transpose matrix
    for i in range(num_lists_in_transpose): 
        list_for_transpose = [] # begin building each transpose list
        for original_list in matrix: 
            # index the element from the non-tranpose list and place it 
            # in the tranposed list corresponding to the index
            list_for_transpose.append(original_list[i])
        transpose_matrix.append(list_for_transpose)
    return transpose_matrix

In [127]:
try:
  curr_adj_mat = time_var_adj[:, :, 0]
  curr_adj_mat = curr_adj_mat/np.sum(curr_adj_mat)
  print('Ho')
except:
  pass

Ho


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
num_age_groups = time_var_adj.shape[2] # years of longitudinal data
upper_tri_indices = np.triu_indices(20, k = 1)
edges_across_time = []
for idx in range(num_age_groups):
  curr_adj_mat = time_var_adj[:, :, idx]
  if np.sum(curr_adj_mat) != 0:
    curr_adj_mat = curr_adj_mat/np.sum(curr_adj_mat) # normalize
  edges_across_time.append(curr_adj_mat[upper_tri_indices])
edges_across_time = transpose(edges_across_time)

In [131]:
all_variances = [np.nanvar(lst) for lst in edges_across_time]
ranked = np.argsort(all_variances)
high_variance_edge_IDs = ranked[::-1][:len(ranked)]

row_indices = upper_tri_indices[0]
col_indices = upper_tri_indices[1]
all_edges = []
for idx in range(len(row_indices)):
  all_edges.append(tuple([med_names[row_indices[idx]], med_names[col_indices[idx]]]))

high_variance_edges = []
for edge_ID in high_variance_edge_IDs:
  high_variance_edges.append([edge_ID, all_edges[edge_ID], all_variances[edge_ID]])

In [132]:
high_variance_edges[:10]

[[0, ('LEV', 'OXC'), 0.0022329912486952507],
 [78, ('VPA', 'TPM'), 0.0007093561923787698],
 [1, ('LEV', 'LTG'), 0.0004112928825035262],
 [2, ('LEV', 'PHB'), 0.0001242499482855136],
 [12, ('LEV', 'TPM'), 0.00011240867015522202],
 [3, ('LEV', 'VPA'), 9.390700955526457e-05],
 [59, ('PHB', 'ZNS'), 5.2976700528041834e-05],
 [14, ('LEV', 'GPT'), 5.231277558404785e-05],
 [38, ('LTG', 'VPA'), 4.936512442797508e-05],
 [63, ('PHB', 'TPM'), 4.627684194480148e-05]]